In [0]:
import os
import sys

sys.path.append(f"{os.getcwd()}/..")

In [0]:
import json

settings = json.loads("""
{
    "read_function": "functions.stream_read_cloudfiles",
    "transform_function": "functions.bronze_standard_transform",
    "write_function": "functions.stream_write_table",
    "dst_table_name": "edsm.bronze.powerPlay",
    "build_history": "true",
    "readStream_load": "/Volumes/edsm/bronze/landing/",
    "readStreamOptions": {
        "cloudFiles.format": "json",
        "pathGlobFilter": "powerPlay.json",
        "cloudFiles.inferColumnTypes": "false",
        "inferSchema": "false",
        "cloudFiles.schemaLocation": "/Volumes/edsm/bronze/utility/powerPlay/_schema/",
        "multiline": "false",
        "badRecordsPath": "/Volumes/edsm/bronze/utility/powerPlay/_badRecords/",
        "cloudFiles.rescuedDataColumn": "_rescued_data"
    },
    "writeStream_format": "delta",
    "writeStreamOptions": {
        "mergeSchema": "false",
        "checkpointLocation": "/Volumes/edsm/bronze/utility/powerPlay/_checkpoints/",
        "delta.columnMapping.mode": "name"
    },
    "writeStream_outputMode": "append",
    "file_schema": {
        "fields": [
            {
                "metadata": {},
                "name": "allegiance",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "coords",
                "nullable": true,
                "type": {
                    "fields": [
                        {
                            "metadata": {},
                            "name": "x",
                            "nullable": true,
                            "type": "double"
                        },
                        {
                            "metadata": {},
                            "name": "y",
                            "nullable": true,
                            "type": "double"
                        },
                        {
                            "metadata": {},
                            "name": "z",
                            "nullable": true,
                            "type": "double"
                        }
                    ],
                    "type": "struct"
                }
            },
            {
                "metadata": {},
                "name": "date",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "government",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "id",
                "nullable": true,
                "type": "long"
            },
            {
                "metadata": {},
                "name": "id64",
                "nullable": true,
                "type": "long"
            },
            {
                "metadata": {},
                "name": "name",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "power",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "powerState",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "state",
                "nullable": true,
                "type": "string"
            },
            {
                "metadata": {},
                "name": "_rescued_data",
                "nullable": true,
                "type": "string"
            }
        ],
        "type": "struct"
    }
}
""")

In [0]:
from functions import bronze_standard_transform

df = (
    spark.read
    .option("multiline", "false")
    .json("/Volumes/edsm/bronze/landing/20250626/powerPlay.json")
    .transform(bronze_standard_transform, settings, spark)
)
df.show(truncate=False)